In [ ]:
import datasets

import nibabel as nib
import os
import pickle
import pandas as pd
import seaborn as sns
import numpy as np

import draw_results
import gene_analysis
import removing_confound
import meta_roi
import meta_voxel
import meta_vertex
import meta_confound
import correlation
from mask import Mask, NiiMask
import mixed_lm

#Assume all data is organized
#Load dataset
centers = datasets.load_centers_all()

#Define ROI Mask
mask_path = './data/mask/rBN_Atlas_246_1mm.nii'
mask = NiiMask(mask_path)

label_pairs = [(2,0), (2,1), (1,0)]

# Create ROI csv, and remove confound for each center

In [ ]:
"""
#Create ROI GMV/CT csv file
for center in centers:
    center.create_rgmv_csv(mask, gmv_csv_prefix='roi_gmv/{}.csv')
    center.create_rct_csv()
"""
#Remove confound
# ROI
removing_confound.remove_roi(centers)
removing_confound.remove_roi(centers, csv_prefix='roi_ct/{}.csv',
                                out_prefix='roi_ct_removed/{}.csv')
"""
# Voxel
centers = datasets.load_centers_edsd()
removing_confound.remove_nii(centers, mask, batch_reg=False)
centers = datasets.load_centers_mcad()
removing_confound.remove_nii(centers, mask, batch_reg=False)
centers = datasets.load_centers_adni()
removing_confound.remove_nii(centers, mask, batch_reg=True, batch_size=4000000)

# Vertex
centers = datasets.load_centers_all()
removing_confound.remove_gii(centers)
"""

# Perform meta-analysis

## ROI

In [ ]:
# ROI
csv_prefixs = ['roi_gmv' ,'roi_ct', 'roi_gmv_removed', 'roi_ct_removed']

csv_dir = './data/meta_csv'
out_dir = './results/meta'
"""
# Create csv for meta analysis
for csv_prefix in csv_prefixs:
    for label_pair in label_pairs:
        label_eg = label_pair[0]
        label_cg = label_pair[1]
        meta_roi.create_csv_for_meta(centers, label_eg, label_cg, csv_prefix, out_path=csv_dir)
"""
#%%
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    # GMV meta-analysis
    meta_roi.meta_gmv(label_eg, label_cg, mask, csv_prefix='roi_gmv', csv_dir=csv_dir, out_dir=out_dir)
    meta_roi.meta_gmv(label_eg, label_cg, mask, csv_prefix='roi_gmv_removed', csv_dir=csv_dir, out_dir=out_dir)
    # CT meta-analysis
    meta_roi.meta_ct(label_eg, label_cg, csv_prefix='roi_ct', mask=mask, save_gii=False, save_nii=True, csv_dir=csv_dir, out_dir=out_dir)
    meta_roi.meta_ct(label_eg, label_cg, csv_prefix='roi_ct_removed', mask=mask, save_gii=False, save_nii=True, csv_dir=csv_dir, out_dir=out_dir)

In [ ]:
csv_dir = './data/meta_csv'
out_dir = './results/meta'
results = meta_roi.meta_gmv(2, 0, mask, csv_prefix='roi_gmv_removed', csv_dir=csv_dir, out_dir=out_dir, save_nii=False)

In [ ]:
df_dict = {}

for k, result in results.items():
    row = [result.total_effect_size, result.total_variance, result.total_standard_error,
            result.total_lower_limit, result.total_upper_limit, result.q, result.z, result.p]
    df_dict[k] = row

df = pd.DataFrame.from_dict(df_dict, orient='index',
                       columns=['total_effect_size', 'total_variance', 'total_standard_error', 'total_lower_limit', 
                        'total_upper_limit', 'q', 'z', 'p'])
df.to_csv('./results/NC_AD_BN.csv')

# Plot correlation between each center

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Create Dataframe with full sites
es_df_columns = ['Total']
for center in centers:
    es_df_columns.append(center.name)

roi_count = 246
es_df = pd.DataFrame(columns=es_df_columns, index=[i for i in range(1, roi_count+1)])

# perform meta_analysis
result_models = meta_roi.meta_gmv(2, 0, mask, save_nii=False,
                                  csv_prefix='roi_gmv_removed')
for roi in range(1, roi_count+1):
    values = [result_models[roi].total_effect_size]
    for center in centers:
        value = np.NaN
        for study, es in zip(result_models[roi].studies, result_models[roi].effect_sizes):
            if center.name == study.name:
                value = es
        values.append(value)
    es_df.loc[roi] = values
es_df = es_df.astype(np.float32)
es_df = es_df.dropna(axis=1, how='any')

corr_df = es_df.corr()
# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(9, 9))
cmap = sns.color_palette("cubehelix_r", as_cmap=True)
# Draw the heatmap with the mask and correct aspect ratio
ax = sns.heatmap(corr_df, cmap='YlGnBu', vmin=0, vmax=1,
            square=True, linewidths=.5, cbar_kws={"shrink": .8})
ax.axis('off')
plt.show()

# Center heterogeneity

In [ ]:
label_eg = 2
label_cg = 0
# GMV meta-analysis
result_models = meta_roi.meta_gmv(label_eg, label_cg, mask,
                                 save_nii=False,csv_prefix='roi_gmv_removed')
from draw_results import plot_correlation
ess = []
heters = []
for k, model in result_models.items():
    ess.append(model.total_effect_size)
    heters.append(model.q)
plot_correlation(ess, heters)

In [ ]:
# Check how many ROI pass correction
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    # GMV meta-analysis
    roi_models = meta_roi.meta_gmv(label_eg, label_cg, mask, csv_prefix='roi_gmv_removed')
    #roi_models = meta_roi.meta_ct(label_eg, label_cg, csv_prefix='roi_ct_removed', mask=mask, save_gii=False, save_nii=False)
    i = 0
    for key, model in roi_models.items():
        if model.p < 0.001/246:
            i += 1
    print(i)

In [ ]:
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    # CT meta-analysis
    roi_models = meta_roi.meta_ct(label_eg, label_cg, csv_prefix='roi_ct_removed', mask=mask, save_gii=False, save_nii=False)
    i = 0
    for key, model in roi_models.items():
        if model.p < 0.001/246:
            i += 1
    print(i)

## Voxel and Vertex

In [ ]:
# Voxel
# Create center mean std nii file
labels = [0, 1, 2]

for center in centers:
    temp_nii = nib.load('./data/mask/save_temp.nii')
    for label in labels:
        center.create_stat_nii(label, temp_nii, nii_dir='mri_smoothed')

# perform meta-analysis
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    meta_voxel.meta_nii(centers, label_eg, label_cg, mri_dir='mri_smoothed')

In [ ]:
"""
# Vertex
# Create center mean std numpy file
labels = [0,1,2]
for center in centers:
    for label in labels:
        center.create_stat_gii(label, prefix='surf/s15.mesh.thickness.resampled_32k.{}.gii')
"""
# perform meta-analysis for resampled surface
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    meta_vertex.meta_gii(centers, label_eg, label_cg)

# personal score

In [ ]:
from data_harmonization import neurocombat

neurocombat(centers, personal_info_prefix = 'personal_info/{}.csv',
                cat_prefix = 'report/cat_{}.xml',
                csv_prefix = 'roi_gmv/{}.csv',
                out_prefix = 'neurocombat_gmv/{}.csv')
neurocombat(centers, personal_info_prefix = 'personal_info/{}.csv',
                cat_prefix = 'report/cat_{}.xml',
                csv_prefix = 'roi_ct/{}.csv',
                out_prefix = 'neurocombat_ct/{}.csv')

In [ ]:
from data_harmonization import neurocombat2

neurocombat2(centers, personal_info_prefix = 'personal_info/{}.csv',
                cat_prefix = 'report/cat_{}.xml',
                csv_prefix = 'roi_gmv/{}.csv',
                out_prefix = 'neurocombat_gmv2/{}.csv',
                gmv=True)
neurocombat2(centers, personal_info_prefix = 'personal_info/{}.csv',
                cat_prefix = 'report/cat_{}.xml',
                csv_prefix = 'roi_ct/{}.csv',
                out_prefix = 'neurocombat_ct2/{}.csv',
                gmv=False)

In [ ]:
# Create stats csv for each label
from personal_scores import create_stats_csv, cal_personal_scores

create_stats_csv(centers, 'neurocombat_gmv', out_dir='./data/stats', divide_tiv=False)
cal_personal_scores(centers, stats_dir='./data/stats/neurocombat_gmv',
                     csv_prefix='neurocombat_gmv/{}.csv', out_prefix='personal_scores/{}.csv')


In [ ]:
from personal_scores import plot_stats
ax, legends = plot_stats('./data/stats/roi_gmv', 217)
import matplotlib.pyplot as plt
ax.legend(legends)
plt.show()

### Subtype based on personal scores -> see subtype_unsupervised.py

## Confound

In [ ]:
# Create csv for meta analysis
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    meta_confound.create_csv_for_meta(centers, label_eg, label_cg)

for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    meta_confound.meta_confound(label_eg, label_cg)

# PET: ABeta

In [ ]:
# abeta PET correlation
from abeta_pet import ttest_by_label
import utils
out_dir_prefix = './results_0401/correlation/{}_{}'

for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    t, p = ttest_by_label(label_eg, label_cg)

    roi_models = meta_roi.meta_gmv(label_eg, label_cg, mask, save_nii=False)
    es1 = [v.total_effect_size for k,v in sorted(roi_models.items())]
    es2 = [v for k,v in sorted(t.items())]
    draw_results.plot_correlation_joint(es1, es2,
        'Effect sizes of ROI GMV', 't-values of ROI Aβ', fontsize=18, show=False,
        save=True, out_path=os.path.join(out_dir_prefix.format(label_eg, label_cg), 'gmv', 'abeta.png'))

    roi_models = meta_roi.meta_ct(label_eg, label_cg, mask, save_nii=False, save_gii=False)
    es1 = [v.total_effect_size for k,v in sorted(roi_models.items()) if int(k)<211]
    es2 = [v for k,v in sorted(t.items()) if int(k)<211]
    draw_results.plot_correlation_joint(es1, es2,
        'Effect sizes of ROI CT', 't-values of ROI Aβ', fontsize=18, show=False,
        save=True, out_path=os.path.join(out_dir_prefix.format(label_eg, label_cg), 'ct', 'abeta.png'))

In [ ]:
from abeta_pet import create_subject_df 
df = create_subject_df()
len(df)

# PET: FDG

In [ ]:
from pet_fdg import create_sum, create_subject_df
subject_df = create_subject_df()

mask = NiiMask(r'I:\data\atlas\grey_matter\brainnetome\BN_Atlas_246_1mm.nii.gz')
create_sum(subject_df, mask)

In [ ]:
# Generate FDG age
import pandas as pd

df1 = pd.read_csv(r'G:\workspace\AD_meta\data\PET\fdg\all_info_fixed.csv', index_col=['PTID', 'VISCODE'])
df2 = pd.read_csv(r'G:\data\dataset_info\AD\ADNI\ADNIMERGE.csv', index_col=['PTID', 'VISCODE'])
true_ages = []
MMSEs = []
for key, row in df1.iterrows():
    row2 = df2.loc[key]
    true_age = row2['AGE'] + row2['Years_bl']
    true_ages.append(true_age)
    MMSEs.append(row2['MMSE'])
df1['TrueAge'] = true_ages
df1['MMSE'] = MMSEs

df1.to_csv(r'G:\workspace\AD_meta\data\PET\fdg\all_info_fixed_age.csv')

In [ ]:
df = pd.read_csv(r'G:\workspace\AD_meta\data\PET\fdg\all_info_fixed_age.csv', index_col='PTID')

files = os.listdir(r'G:\workspace\AD_meta\data\PET\fdg\nii')
files = [f[:-4] for f in files if 'nii' in f]

sub_df = df[df.index.isin(files)]
sub_df.to_csv(r'G:\workspace\AD_meta\data\PET\fdg\all_info_fixed_age_flitered.csv')

In [ ]:
from pet_fdg import ttest_by_label
out_dir_prefix = './results_0401/correlation/{}_{}'

for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    t, p = ttest_by_label(label_eg, label_cg)

    roi_models = meta_roi.meta_gmv(label_eg, label_cg, mask, save_nii=False)
    es1 = [v.total_effect_size for k,v in sorted(roi_models.items())]
    es2 = [v for k,v in sorted(t.items())]
    draw_results.plot_correlation_joint(es1, es2,
        'Effect sizes of ROI GMV', 't-values of ROI FDG', fontsize=18, show=False,
        save=True, out_path=os.path.join(out_dir_prefix.format(label_eg, label_cg), 'gmv', 'FDG.png'))
    
    roi_models = meta_roi.meta_ct(label_eg, label_cg, mask, save_nii=False, save_gii=False)
    es1 = [v.total_effect_size for k,v in sorted(roi_models.items()) if int(k)<211]
    es2 = [v for k,v in sorted(t.items()) if int(k)<211]
    draw_results.plot_correlation_joint(es1, es2, 
        'Effect sizes of ROI CT', 't-values of ROI FDG', fontsize=18, show=False,
        save=True, out_path=os.path.join(out_dir_prefix.format(label_eg, label_cg), 'ct', 'FDG.png'))

In [ ]:
from nilearn import plotting

_dir = r'G:\workspace\AD_meta\data\PET\fdg\nii'
files = os.listdir(_dir)
for f in files:
    if '.nii' in f:
        nii = nib.load(os.path.join(_dir, f))
        plotting.plot_stat_map(nii, output_file=f'./tmp/{f}.png')

In [ ]:
from pet_fdg import create_subject_df 
df = create_subject_df()
len(df)

# PET:Juspace

In [ ]:
pet_dir = './data/PET/masked_mean'
out_dir_prefix = './results/correlation/{}_{}/{}/PET'
labels = ['NC', 'MCI', 'AD']

df1 = pd.DataFrame(columns = ['x', 'y', 'Pearson\'s r', 'Size', 'p'])
x_labels = []
y_labels = []
label_pairs = [(1,0), (2,1), (2,0)]

y = 1
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    roi_gmv_models = meta_roi.meta_gmv(label_eg, label_cg, mask, save_nii=False)
    roi_ct_models = meta_roi.meta_ct(label_eg, label_cg, save_gii=False, save_nii=False)

    gmv_out_dir = out_dir_prefix.format(label_eg, label_cg, 'gmv')
    ct_out_dir = out_dir_prefix.format(label_eg, label_cg, 'ct')
    if not os.path.exists(gmv_out_dir):
        os.mkdir(gmv_out_dir)
    if not os.path.exists(ct_out_dir):
        os.mkdir(ct_out_dir)

    gmv_results = correlation.cor_roi_pet(roi_gmv_models, pet_dir, out_dir=gmv_out_dir,
                                        fig_width=5, fig_height=5,
                                        fontsize=18, save=True, show=False)
    ct_results = correlation.cor_roi_pet(roi_ct_models, pet_dir, out_dir=ct_out_dir,
                                        fig_width=5, fig_height=5,
                                        fontsize=18,  save=True, show=False)
    if not x_labels:
        for result in gmv_results:
            x_labels.append(result.name)
    y_labels.append('{}_{}_CT'.format(labels[label_eg], labels[label_cg]))
    y_labels.append('{}_{}_GMV'.format(labels[label_eg], labels[label_cg]))
    
    x = 1
    for result in ct_results:
        df1 = df1.append({'x' : x, 'y' : y, 'Pearson\'s r' : result.r,
                        'Size':abs(result.r), 'p': result.p, '-log10 p':-np.log10(result.p)}, 
                        ignore_index = True) 
        x += 1

    x = 1
    for result in gmv_results:
        df1 = df1.append({'x' : x, 'y' : y+1, 'Pearson\'s r' : result.r,
                        'Size':abs(result.r), 'p': result.p, '-log10 p':-np.log10(result.p)}, 
                        ignore_index = True) 
        x += 1
    y += 2

In [ ]:
import abeta_pet
import pet_fdg
import correlation

labels = ['NC', 'MCI', 'AD']
y_labels = []
df2 = pd.DataFrame(columns = ['x', 'y', 'Pearson\'s r', 'Size', 'p'])
label_pairs = [(1,0), (2,1), (2,0)]
i = 0
y = 1
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    roi_gmv_models = meta_roi.meta_gmv(label_eg, label_cg, mask, save_nii=False)
    roi_ct_models = meta_roi.meta_ct(label_eg, label_cg, save_gii=False, save_nii=False)

    gmv_es_dict = {}
    for key, roi_gmv_model in roi_gmv_models.items():
        gmv_es_dict[key] = roi_gmv_model.total_effect_size
        
    ct_es_dict = {}
    for key, roi_ct_model in roi_ct_models.items():
        ct_es_dict[key] = roi_ct_model.total_effect_size

    abeta_t, _ = abeta_pet.ttest_by_label(label_eg, label_cg)
    fdg_t, _ =  pet_fdg.ttest_by_label(label_eg, label_cg)
    
    gmv_results = [correlation.pearson_r('Abeta', gmv_es_dict, abeta_t), 
                   correlation.pearson_r('FDG', gmv_es_dict, fdg_t)]
    ct_results = [correlation.pearson_r('Abeta', ct_es_dict, abeta_t), 
                   correlation.pearson_r('FDG', ct_es_dict, fdg_t)]

    if i == 0:
        for result in gmv_results:
            x_labels.append(result.name)
        i += 1
    y_labels.append('{}_{}_CT'.format(labels[label_eg], labels[label_cg]))
    y_labels.append('{}_{}_GMV'.format(labels[label_eg], labels[label_cg]))
    
    x = 13
    for result in ct_results:
        df2 = df2.append({'x' : x, 'y' : y, 'Pearson\'s r' : result.r,
                        'Size':abs(result.r), 'p': result.p, '-log10 p':-np.log10(result.p)}, 
                        ignore_index = True)
        x += 1

    x = 13
    for result in gmv_results:
        df2 = df2.append({'x' : x, 'y' : y+1, 'Pearson\'s r' : result.r,
                        'Size':abs(result.r), 'p': result.p, '-log10 p':-np.log10(result.p)}, 
                        ignore_index = True)
        x += 1
    y += 2

In [ ]:
df = pd.concat([df1, df2], ignore_index=True, sort=False)
df = df.append({'x' : 1, 'y' : 0, 'Pearson\'s r' : 0,
                        'Size':0, 'p': 0, '-log10 p':0}, 
                        ignore_index = True)
df = df.append({'x' : 1, 'y' : 7, 'Pearson\'s r' : 0,
                        'Size':0, 'p': 0, '-log10 p':0}, 
                        ignore_index = True)

In [ ]:
import matplotlib.pyplot as plt
def draw_pet_relplot(df, x_labels, y_labels):
    # Draw each cell as a scatter point with varying size and color
    sns.set_theme(style="whitegrid")
    g = sns.relplot(
        data=df, height=3.7, aspect=1.8,
        x="x", y="y", hue="Pearson\'s r", size="-log10 p",
        palette="vlag", hue_norm=(-0.8, 0.8), edgecolor=".7",
        sizes=(0, 600), size_norm=(1, 15),
        legend='full')

    g.ax.set_xticks(np.arange(1, len(x_labels)+1))
    g.ax.set_xticklabels(x_labels)

    g.ax.set_yticks(np.arange(1, len(y_labels)+1))
    g.ax.set_yticklabels(y_labels)
    
    g.despine(left=True, bottom=True)

    for label in g.ax.get_xticklabels():
        label.set_rotation(90)
        x, y = label.get_position()
        label.set_position((x, y-0.2))
    for artist in g.legend.legendHandles:
        artist.set_edgecolor(".7")

    #plt.savefig(r'D:\Desktop\abeta_fdg.png', dpi=600)
    plt.show()

draw_pet_relplot(df, x_labels, y_labels)

# PLSR with Gene

In [ ]:
n_perm_boot = 5000
n_components = 5
out_dir_prefix ='./results/gene/{}_{}'
label_pairs = [(2,0)]
for label_pair in label_pairs:
    label_eg = label_pair[0]
    label_cg = label_pair[1]
    out_dir = out_dir_prefix.format(label_eg, label_cg)
    if not os.path.exists(out_dir):
        os.mkdir(out_dir)
    roi_gmv_models = meta_roi.meta_gmv(label_eg, label_cg, mask, save_nii=False)
    roi_ct_models = meta_roi.meta_ct(label_eg, label_cg, save_gii=False, save_nii=False)
    
    gmv_es_dict = {}
    for k,v in sorted(roi_gmv_models.items()):
        gmv_es_dict[int(k)] = v.total_effect_size
        
    ct_es_dict = {}
    for k,v in sorted(roi_ct_models.items()):
        ct_es_dict[int(k)] = v.total_effect_size
    print('model calculated')
    gmv_plsr = gene_analysis.plsr(gmv_es_dict, n_components=n_components,
                                  n_perm=n_perm_boot, n_boot=n_perm_boot,
                                  out_path=os.path.join(out_dir, 'plsr_gmv.csv'))
    print('GMV Finished')
    ct_plsr = gene_analysis.plsr(ct_es_dict, n_components=n_components,
                                  n_perm=n_perm_boot, n_boot=n_perm_boot,
                                  out_path=os.path.join(out_dir, 'plsr_ct.csv'))

    with open(os.path.join(out_dir, 'plsr_gmv.pickle'), 'wb') as f:
        pickle.dump(gmv_plsr, f)
    with open(os.path.join(out_dir, 'plsr_ct.pickle'), 'wb') as f:
        pickle.dump(ct_plsr, f)

## Check PLSR models

In [ ]:
# Check plsr model
result_plsr_dir = './results/gene/2_0'
with open(os.path.join(result_plsr_dir, 'plsr_gmv.pickle'), 'rb') as f:
    gmv_plsr = pickle.load(f)
with open(os.path.join(result_plsr_dir, 'plsr_ct.pickle'), 'rb') as f:
    ct_plsr = pickle.load(f)
    
print('GMV: variance_explained:{:.2f} ,p:{:.2e}'.format(gmv_plsr.varexp[0] , gmv_plsr.permres.pvals[0]))
print('CT: variance_explained:{:.2f} ,p:{:.2e}'.format(ct_plsr.varexp[0] , ct_plsr.permres.pvals[0]))

In [ ]:
gmv_plsr.permres.pvals

# Robustness of PLSR

In [ ]:
# Take 80% subjects for meta then performed PLSR
label_eg = 2
label_cg = 0
for i in range(5000):
    roi_gmv_models = meta_roi.create_csv_for_meta(centers, label_eg, label_cg,
                                    csv_prefix='roi_gmv_removed',
                                    out_path=os.path.join('./data/meta_csv/robust', str(i)),
                                    ratio=0.8)

In [ ]:
for i in range(5000):
    print(i)
    roi_gmv_models = meta_roi.meta_gmv(2, 0, mask,
                                    csv_dir=f'./data/meta_csv/robust/{i}',
                                    out_dir='./results/robust',
                                    save_nii=False)
    with open(f'./results/robust/result_models_{i}.pkl', 'wb') as f:
        pickle.dump(roi_gmv_models, f)

In [ ]:
from scipy.stats import pearsonr
import pickle

main_models = meta_roi.meta_gmv(2, 0, mask, csv_prefix='roi_gmv_removed')
main_ess = [model.total_effect_size for k, model in main_models.items()]
rs = []
for i in range(5000):
    with open(f'./results/robust/result_models_{i}.pkl', 'rb') as f:
        roi_gmv_models = pickle.load(f)
        ess = [model.total_effect_size for k,model in roi_gmv_models.items()]
        r,p = pearsonr(main_ess, ess)
        rs.append(r)
with open(f'./results/robust/rs.pkl', 'wb') as f:
    pickle.dump(rs, f)

In [ ]:
import pickle
with open(f'./results/robust/rs.pkl', 'rb') as f:
    rs = pickle.load(f)
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(x=rs, fill=True,
            alpha=.5, linewidth=0,)
plt.show()

In [ ]:
from scipy.stats import pearsonr
import pandas as pd
df = pd.read_csv('./results/gene/2_0/plsr_gmv.csv', index_col=0)
values = df['pls1'].values
rs = []
for i in range(5000):
    print(i)
    tmp_df = pd.read_csv(f'./results/robust/plsr/plsr_gmv_{i}.csv', index_col=0)
    tmp_values = tmp_df['pls1'].values
    r,p = pearsonr(values, tmp_values)
    rs.append(r)
import pickle
with open(f'./results/robust/rs_plsr.pkl', 'wb') as f:
    pickle.dump(rs, f)

In [ ]:
import pickle
import numpy as np
with open(f'./results/robust/rs_plsr.pkl', 'rb') as f:
    rs = pickle.load(f)
rs = np.abs(rs)
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(x=rs, fill=True,
            alpha=.5, linewidth=0,)
plt.show()

# Personal Scores & Subtypes
## !! deprecated !!

In [ ]:
from data_harmonization import neurocombat
neurocombat(centers, personal_info_prefix = 'personal_info/{}.csv',
                cat_prefix = 'report/cat_{}.xml',
                csv_prefix = 'roi_ct/{}.csv',
                out_prefix = 'neurocombat_ct/{}.csv')

In [ ]:
# Use harmonized data
# Already Controlled For Age Gender TIV
from personal_scores import create_stats_csv
create_stats_csv(centers, 'neurocombat_gmv', out_dir='./data/stats/', divide_tiv=False)
create_stats_csv(centers, 'neurocombat_ct', out_dir='./data/stats/', divide_tiv=False)

In [ ]:
import personal_scores
import importlib
importlib.reload(personal_scores)
personal_scores.cal_personal_scores(centers, './data/stats/neurocombat_gmv',
                    'neurocombat_gmv/{}.csv', 'personal_scores/{}.csv')

In [ ]:
import personal_scores
import importlib
importlib.reload(personal_scores)
personal_scores.cal_personal_scores(centers, './data/stats/neurocombat_ct',
                    'neurocombat_ct/{}.csv', 'personal_scores_ct/{}.csv')

## Subtype analysis see subtype_unsupervised.py

In [ ]:
from draw_center_summary import get_center_stats
get_center_stats(csv_path='./results/s.csv')

# Personal Scores & Subtypes
## Using Gaussian process Regression
### see normative.ipynb and personal_score_anlaysis.ipnb